In [1]:
import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()

DB_URI = os.getenv('DB_URI_DEV')

In [22]:
df = pd.read_sql('articles', DB_URI, index_col = 'id')

In [23]:
df.sample(10)

,url,title,date,body,author
id,,,,,
8859,https://www.inform.kz/ru/ceny-na-yayca-v-evrop...,Цены на яйца в Европе взлетели на 76%,2012-04-05,АСТАНА. 5 апреля. КАЗИНФОРМ - Цены на куриные ...,None
25298,https://www.inform.kz/ru/kazakstan-barysy-sere...,«Қазақстан барысы»: Серебряный пояс и 30 тыс. ...,2012-09-23,АСТАНА. 23 сентября. КАЗИНФОРМ - В субботу опр...,None
11556,https://www.inform.kz/ru/k-20-letiyu-vooruzhen...,К 20-летию Вооруженных сил РК выпущен сборник ...,2012-05-02,АСТАНА. 2 мая. КАЗИНФОРМ - В Астане состоялась...,None
26440,https://www.inform.kz/ru/srochno-akim-atyrausk...,СРОЧНО: Аким Атырауской области заявил о корру...,2012-10-04,АТЫРАУ. 4 октября. КАЗИНФОРМ ­- В Атырауской о...,Виктор Сутягин
16959,https://www.inform.kz/ru/finpoliciya-rk-i-cent...,Финполиция РК и Центр ОБСЕ в Астане проведут н...,2012-06-26,АСТАНА. 26 июня. КАЗИНФОРМ - Агентством по бор...,None
28309,https://www.inform.kz/ru/v-almatinskoy-oblasti...,В Алматинской области пройдет «День опекуна»,2012-10-23,ТАЛДЫКОРГАН. 23 октября. КАЗИНФОРМ - Департаме...,Сандугаш Дуйсенова
30873,https://www.inform.kz/ru/v-kazahstane-vdol-mag...,В Казахстане вдоль магистрали «Западная Европа...,2012-11-20,АСТАНА. 20 ноября. КАЗИНФОРМ - Вдоль междунаро...,None
29598,https://www.inform.kz/ru/srochno-v-vko-proizos...,СРОЧНО: В ВКО произошло землетрясение,2012-11-07,УСТЬ-КАМЕНОГОРСК. 7 ноября. КАЗИНФОРМ - Сегодн...,Людмила Малько
17485,https://www.inform.kz/ru/v-rezul-tate-popytki-...,В результате попытки угона пассажирского самол...,2012-07-01,ПЕКИН. 1 июля. КАЗИНФОРМ - В результате попытк...,Руслан Сулейменов


# Text preprocessing
## Basic preprocessing
* Lower-case
* Remove numbers and non-letter characters
* Lemmatization

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from pymystem3 import Mystem
mystem = Mystem()

tfidf = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.9,
    min_df = 3,
    max_features = 5000,
)

In [26]:
def preprocess(text):
    global mystem
    text = text.lower()
    text = re.sub('[\W_\d]+', ' ', text)
    text = mystem.lemmatize(text)
    text = ''.join(text)
    return text            

df['body_preprocessed'] = df.body.progress_apply(preprocess)

  0%|          | 0/32265 [00:00<?, ?it/s]

In [29]:
tfidf.fit(df.body_preprocessed)

TfidfVectorizer(max_df=0.9, max_features=5000, min_df=3, ngram_range=(1, 2))

In [48]:
n, top_n = 5, 10
a_ind = df.sample(n).index
articles = df.body_preprocessed[a_ind]
def top_n_tokens(tfidf: TfidfVectorizer, articles: pd.Series, top_n: int) -> list:
    a_tfidf = tfidf.transform(articles).todense().A
    a_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :n]
    m = tfidf.get_feature_names()
    
    a_tfidf = tfidf.transform(articles).todense().A
    a_ind_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :top_n]
    fn = tfidf.get_feature_names()

    return [(indT, [(fn[i], a_tfidf[ind, i]) for i in a_ind_srt[ind]]) for ind, indT in enumerate(articles.index)]

top_n_tokens(tfidf, articles, top_n)

[(28242,
  [('язык', 0.44022409020221287),
   ('ассамблея народ', 0.23546544081170928),
   ('проект', 0.21657405680601421),
   ('ассамблея', 0.20080949489782435),
   ('народ казахстан', 0.1924098240117017),
   ('народ', 0.14966612224884104),
   ('казахстан', 0.1357176647213042),
   ('обучение', 0.13305423557765492),
   ('банк развитие', 0.13049758314326018),
   ('данный проект', 0.12588624056029157)]),
 (24459,
  [('га', 0.45981556349634106),
   ('лес', 0.43116012901456197),
   ('лесной', 0.3437506478136169),
   ('тыс', 0.19751451809946174),
   ('посадка', 0.13981318615610086),
   ('запрет', 0.12851349294471548),
   ('объем', 0.12248731955273191),
   ('комитет', 0.12172398159260392),
   ('область', 0.1161618069159607),
   ('за последний', 0.10901193283363084)]),
 (65,
  [('таджикистан', 0.38833448174915963),
   ('религиозный', 0.23059020619533108),
   ('празднование', 0.21672493099298382),
   ('мороз', 0.18328177218667951),
   ('новый', 0.1801535865772621),
   ('мусульманин', 0.1718366

In [31]:
n, top_n = 5, 10
a_ind = df.sample(n).index
a_tfidf = tfidf.transform(df.body_preprocessed[a_ind]).todense().A
a_ind_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :top_n]

fn = tfidf.get_feature_names()
for i, row in enumerate(a_ind_srt):
    print(f'Word: weight')
    for j in row:
        print(f'{fn[j]}: {a_tfidf[i, j]}') 

Word: weight
тараза: 0.3366082990312772
город: 0.2426252993265763
спортивный: 0.20158440171422795
аким: 0.16002007723596015
торжественный открытие: 0.14309020631057098
так что: 0.1414247162028668
развлекательный: 0.14102584426677844
празднование день: 0.14063351151022835
танец: 0.13903433142272884
желающий: 0.1384944943490746
Word: weight
спорт: 0.4495710378893357
школа: 0.24833543016937137
шымкент: 0.24500430489329403
они: 0.21343508324444804
из они: 0.20391915298183486
бассейн: 0.17089052268904492
заниматься: 0.16489348788432975
тыс человек: 0.15105817367419733
вид спорт: 0.15033709698210515
вид: 0.14805982481950364
Word: weight
олимпиада: 0.46153742537951037
республиканский: 0.2544250152977307
гуманитарный: 0.20424781191291638
цикл: 0.1984402771505901
призер: 0.1643565901452187
школьник: 0.1627229667492006
карагандинский: 0.15855585743198602
образование: 0.1406075583670641
грант: 0.14028018386217686
звание: 0.12759369625258704
Word: weight
полуфинал: 0.392612351740661
турнир: 0.2983